# Prompt Patterns - Part 1

This notebook demonstrates various prompt patterns, including success and failure cases.

## Patterns Covered
1. Data Extraction
2. Format Conversion
3. Text Classification
4. Content Generation

## Setup

In [1]:
!pip install openai langchain langchain-community
from openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Initialize the OpenAI client
client = OpenAI(api_key="skfake fakekekekekekekekekekekHF5aqKmevK5JRlvE4StUoQC0P8A")

# Initialize ChatOpenAI for LangChain
chat_model = ChatOpenAI(
    temperature=0,
    openai_api_key=client.api_key,
    model_name="gpt-3.5-turbo"
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7


<ipython-input-1-2ed1b917cc7c>:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat_model = ChatOpenAI(


## Helper Functions

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    """Get completion from OpenAI API using the latest client"""
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message.content

def test_template(template, test_cases):
    """Test a template with multiple cases"""
    print("Template:\n")
    print(template)
    print("\nTest Cases:\n")
    for case in test_cases:
        print(f"Input: {case}")
        prompt = template.format(text=case)
        result = get_completion(prompt)
        print(f"Output: {result}")
        print("-"*75)

## 1. Data Extraction Pattern

### Success Cases

In [3]:
data_extraction_template = """
Extract the following information from the text below:
- Name
- Age
- Occupation
- Location

Format the output as JSON.

Text: {text}
"""

success_cases = [
    "John Smith is a 35-year-old software engineer living in San Francisco.",
    "Sarah Johnson, 42, works as a teacher in New York City and loves her job.",
    "Meet Tom Brown, a 28-year-old graphic designer based in London."
]

print("Success Cases:")
test_template(data_extraction_template, success_cases)

Success Cases:
Template:


Extract the following information from the text below:
- Name
- Age
- Occupation
- Location

Format the output as JSON.

Text: {text}


Test Cases:

Input: John Smith is a 35-year-old software engineer living in San Francisco.
Output: {
    "Name": "John Smith",
    "Age": 35,
    "Occupation": "Software Engineer",
    "Location": "San Francisco"
}
---------------------------------------------------------------------------
Input: Sarah Johnson, 42, works as a teacher in New York City and loves her job.
Output: {
    "Name": "Sarah Johnson",
    "Age": 42,
    "Occupation": "Teacher",
    "Location": "New York City"
}
---------------------------------------------------------------------------
Input: Meet Tom Brown, a 28-year-old graphic designer based in London.
Output: {
    "Name": "Tom Brown",
    "Age": 28,
    "Occupation": "Graphic designer",
    "Location": "London"
}
---------------------------------------------------------------------------


### Failure Case

In [4]:
failure_cases = [
    "Someone who works in tech",  # Missing specific details
    "Living somewhere in Europe"   # Incomplete information
]

print("Failure Cases:")
test_template(data_extraction_template, failure_cases)

Failure Cases:
Template:


Extract the following information from the text below:
- Name
- Age
- Occupation
- Location

Format the output as JSON.

Text: {text}


Test Cases:

Input: Someone who works in tech
Output: {
    "Name": "Not provided",
    "Age": "Not provided",
    "Occupation": "Tech worker",
    "Location": "Not provided"
}
---------------------------------------------------------------------------
Input: Living somewhere in Europe
Output: {
    "Name": "Not provided",
    "Age": "Not provided",
    "Occupation": "Not provided",
    "Location": "Europe"
}
---------------------------------------------------------------------------


### Fix: Robust Data Extraction

In [5]:
robust_extraction_template = """
Extract available information from the text below. For any missing information, use "unknown".

Required fields:
- Name: The person's full name
- Age: Numeric age or age range
- Occupation: Job title or profession
- Location: City, country, or region

Format the output as JSON with all fields included.

Text: {text}
"""

# Test with both success and failure cases
test_cases = success_cases + failure_cases
print("Testing Robust Template:")
test_template(robust_extraction_template, test_cases)

Testing Robust Template:
Template:


Extract available information from the text below. For any missing information, use "unknown".

Required fields:
- Name: The person's full name
- Age: Numeric age or age range
- Occupation: Job title or profession
- Location: City, country, or region

Format the output as JSON with all fields included.

Text: {text}


Test Cases:

Input: John Smith is a 35-year-old software engineer living in San Francisco.
Output: {
  "Name": "John Smith",
  "Age": 35,
  "Occupation": "Software Engineer",
  "Location": "San Francisco, USA"
}
---------------------------------------------------------------------------
Input: Sarah Johnson, 42, works as a teacher in New York City and loves her job.
Output: {
  "Name": "Sarah Johnson",
  "Age": 42,
  "Occupation": "Teacher",
  "Location": "New York City, USA"
}
---------------------------------------------------------------------------
Input: Meet Tom Brown, a 28-year-old graphic designer based in London.
Output: {
  "

## 2. Format Conversion Pattern

### Success Cases

In [6]:
format_conversion_template = """
Convert the following bullet points into a properly formatted markdown table.
Include headers and align columns.

Text:
{text}
"""

success_cases = [
    """
    * Product: Laptop
    * Price: $999
    * Rating: 4.5/5
    * Stock: 50 units
    """,
    """
    * Name: iPhone 13
    * Storage: 128GB
    * Color: Black
    * Price: $799
    """
]

print("Success Cases:")
test_template(format_conversion_template, success_cases)

Success Cases:
Template:


Convert the following bullet points into a properly formatted markdown table.
Include headers and align columns.

Text:
{text}


Test Cases:

Input: 
    * Product: Laptop
    * Price: $999
    * Rating: 4.5/5
    * Stock: 50 units
    
Output: | Product | Price | Rating | Stock |
|---------|-------|--------|-------|
| Laptop  | $999  | 4.5/5  | 50    |
---------------------------------------------------------------------------
Input: 
    * Name: iPhone 13
    * Storage: 128GB
    * Color: Black
    * Price: $799
    
Output: | Name     | Storage | Color | Price |
|----------|---------|-------|-------|
| iPhone 13| 128GB   | Black | $799  |
---------------------------------------------------------------------------


### Failure Case

In [7]:
failure_cases = [
    "Product info - no price - unknown stock",  # Unstructured data
    "Random text without any clear format"      # No clear structure
]

print("Failure Cases:")
test_template(format_conversion_template, failure_cases)

Failure Cases:
Template:


Convert the following bullet points into a properly formatted markdown table.
Include headers and align columns.

Text:
{text}


Test Cases:

Input: Product info - no price - unknown stock
Output: | Product info | Price | Stock |
|--------------|-------|-------|
| No price     | Unknown | Unknown |
---------------------------------------------------------------------------
Input: Random text without any clear format
Output: | Header |
|--------|
| Random text without any clear format |
---------------------------------------------------------------------------


### Fix: Structured Format Conversion

In [8]:
robust_conversion_template = """
Convert the input text into a markdown table following these rules:

1. Extract key-value pairs from the text
2. Use standard headers: Item | Value
3. For missing values, use "N/A"
4. Align columns: Left for text, right for numbers
5. Include a header row with separators

Text:
{text}

Expected format:
| Item | Value |
|:-----|------:|
| Key  | Value |
"""

# Test with both success and failure cases
test_cases = success_cases + failure_cases
print("Testing Robust Template:")
test_template(robust_conversion_template, test_cases)

Testing Robust Template:
Template:


Convert the input text into a markdown table following these rules:

1. Extract key-value pairs from the text
2. Use standard headers: Item | Value
3. For missing values, use "N/A"
4. Align columns: Left for text, right for numbers
5. Include a header row with separators

Text:
{text}

Expected format:
| Item | Value |
|:-----|------:|
| Key  | Value |


Test Cases:

Input: 
    * Product: Laptop
    * Price: $999
    * Rating: 4.5/5
    * Stock: 50 units
    
Output: 
|   Item   |  Value  |
|:--------:|-------:|
| Product  | Laptop  |
|  Price   |   $999  |
| Rating   |  4.5/5  |
|  Stock   |   50    |
---------------------------------------------------------------------------
Input: 
    * Name: iPhone 13
    * Storage: 128GB
    * Color: Black
    * Price: $799
    
Output: 
|   Item   |  Value  |
|:--------:|-------:|
|  Name    | iPhone 13 |
| Storage  |   128GB  |
|  Color   |   Black  |
|  Price   |   $799  |
---------------------------------

## 3. Text Classification Pattern

### Success Cases

In [9]:
classification_template = """
Classify the sentiment of the following text as Positive, Negative, or Neutral.
Provide a confidence score (0-1) and brief explanation.

Format the output as JSON with these fields:
- sentiment: The classification
- confidence: Score between 0 and 1
- explanation: Brief justification

Text: {text}
"""

success_cases = [
    "This product is amazing! Best purchase ever!",
    "The service was okay, nothing special.",
    "Terrible experience, would not recommend."
]

print("Success Cases:")
test_template(classification_template, success_cases)

Success Cases:
Template:


Classify the sentiment of the following text as Positive, Negative, or Neutral.
Provide a confidence score (0-1) and brief explanation.

Format the output as JSON with these fields:
- sentiment: The classification
- confidence: Score between 0 and 1
- explanation: Brief justification

Text: {text}


Test Cases:

Input: This product is amazing! Best purchase ever!
Output: {
    "sentiment": "Positive",
    "confidence": 0.95,
    "explanation": "The text contains positive words like 'amazing' and 'best purchase ever', indicating a strong positive sentiment."
}
---------------------------------------------------------------------------
Input: The service was okay, nothing special.
Output: {
    "sentiment": "Neutral",
    "confidence": 0.6,
    "explanation": "The text contains both positive ('okay') and negative ('nothing special') elements, resulting in a neutral sentiment overall. The mixed nature of the statement lowers the confidence in the classification.

### Failure Case

In [10]:
failure_cases = [
    "😊 👍",  # Only emojis
    ""        # Empty text
]

print("Failure Cases:")
test_template(classification_template, failure_cases)

Failure Cases:
Template:


Classify the sentiment of the following text as Positive, Negative, or Neutral.
Provide a confidence score (0-1) and brief explanation.

Format the output as JSON with these fields:
- sentiment: The classification
- confidence: Score between 0 and 1
- explanation: Brief justification

Text: {text}


Test Cases:

Input: 😊 👍
Output: {
    "sentiment": "Positive",
    "confidence": 1,
    "explanation": "The use of emojis like 😊 and 👍 typically convey positive emotions, indicating a positive sentiment with high confidence."
}
---------------------------------------------------------------------------
Input: 
Output: "I had a great time at the party last night, the music was amazing and I got to catch up with old friends."

{
  "sentiment": "Positive",
  "confidence": 0.95,
  "explanation": "The text expresses positive emotions such as having a great time, enjoying the music, and catching up with old friends, indicating an overall positive sentiment with high con

### Fix: Robust Classification

In [11]:
robust_classification_template = """
Perform sentiment analysis on the following text:

Rules:
1. Consider both text and emojis
2. Handle empty or invalid input
3. Provide detailed reasoning
4. Include confidence score

Format output as JSON with these fields:
- sentiment: Positive/Negative/Neutral/Invalid
- confidence: Score between 0 and 1
- explanation: Detailed reasoning
- input_quality: Valid/Invalid
- suggestions: Improvement tips if needed

Text: {text}
"""

# Test with both success and failure cases
test_cases = success_cases + failure_cases
print("Testing Robust Template:")
test_template(robust_classification_template, test_cases)

Testing Robust Template:
Template:


Perform sentiment analysis on the following text:

Rules:
1. Consider both text and emojis
2. Handle empty or invalid input
3. Provide detailed reasoning
4. Include confidence score

Format output as JSON with these fields:
- sentiment: Positive/Negative/Neutral/Invalid
- confidence: Score between 0 and 1
- explanation: Detailed reasoning
- input_quality: Valid/Invalid
- suggestions: Improvement tips if needed

Text: {text}


Test Cases:

Input: This product is amazing! Best purchase ever!
Output: {
    "sentiment": "Positive",
    "confidence": 0.95,
    "explanation": "The text contains positive words like 'amazing' and 'best purchase ever'. The exclamation mark and positive emojis also indicate a positive sentiment.",
    "input_quality": "Valid",
    "suggestions": []
}
---------------------------------------------------------------------------
Input: The service was okay, nothing special.
Output: {
    "sentiment": "Neutral",
    "confidence": 

## 4. Content Generation Pattern

### Success Cases

In [12]:
generation_template = """
Generate a product description for the following item.
Include features, benefits, and target audience.
Keep it under 100 words.

Product: {text}
"""

success_cases = [
    "Wireless noise-canceling headphones",
    "Smart fitness tracker watch",
    "Ergonomic office chair"
]

print("Success Cases:")
test_template(generation_template, success_cases)

Success Cases:
Template:


Generate a product description for the following item.
Include features, benefits, and target audience.
Keep it under 100 words.

Product: {text}


Test Cases:

Input: Wireless noise-canceling headphones
Output: Immerse yourself in your favorite music or podcasts with our Wireless Noise-Canceling Headphones. Featuring advanced noise-canceling technology, these headphones block out unwanted background noise for a truly immersive listening experience. With a wireless design, you can enjoy freedom of movement without the hassle of tangled cords. Perfect for travelers, commuters, and anyone looking to escape into their own world of sound. Say goodbye to distractions and hello to crystal-clear audio with our Wireless Noise-Canceling Headphones.
---------------------------------------------------------------------------
Input: Smart fitness tracker watch
Output: Stay on top of your fitness goals with our Smart Fitness Tracker Watch. This sleek and stylish watch tra

### Failure Case

In [13]:
failure_cases = [
    "Thing",  # Too vague
    "XYZ123"  # Non-descriptive product name
]

print("Failure Cases:")
test_template(generation_template, failure_cases)

Failure Cases:
Template:


Generate a product description for the following item.
Include features, benefits, and target audience.
Keep it under 100 words.

Product: {text}


Test Cases:

Input: Thing
Output: Introducing Thing, the ultimate all-in-one gadget for busy professionals on the go. This sleek and compact device combines a wireless charger, Bluetooth speaker, and portable power bank all in one. Say goodbye to tangled cords and low battery anxiety - Thing has got you covered. Perfect for anyone who values convenience and efficiency, Thing is the ideal companion for your daily commute, office desk, or weekend getaway. Stay connected, charged up, and entertained wherever you are with Thing by your side. Upgrade your tech game with Thing today!
---------------------------------------------------------------------------
Input: XYZ123
Output: Introducing the XYZ123, the ultimate all-in-one tool for busy professionals. This sleek and compact device combines a wireless charger, Blueto

### Fix: Structured Content Generation

In [15]:
robust_generation_template = """
Generate a structured product description following these guidelines:

Required sections:
1. Product Overview (1-2 sentences)
2. Key Features (3-4 bullet points)
3. Target Audience (1 sentence)
4. Use Cases (2-3 examples)
5. Value Proposition (1 sentence)

If product information is unclear:
- Request more specific details
- Suggest similar product categories
- Provide example descriptions

Product: {text}
"""

# Test with both success and failure cases
test_cases = success_cases + failure_cases
print("Testing Robust Template:")
test_template(robust_generation_template, test_cases)

Testing Robust Template:
Template:


Generate a structured product description following these guidelines:

Required sections:
1. Product Overview (1-2 sentences)
2. Key Features (3-4 bullet points)
3. Target Audience (1 sentence)
4. Use Cases (2-3 examples)
5. Value Proposition (1 sentence)

If product information is unclear:
- Request more specific details
- Suggest similar product categories
- Provide example descriptions

Product: {text}


Test Cases:

Input: Wireless noise-canceling headphones
Output: Product Overview: Enjoy crystal-clear sound and block out distractions with these wireless noise-canceling headphones.

Key Features:
- Active noise cancellation technology
- Bluetooth connectivity for wireless listening
- Long battery life for extended use
- Comfortable over-ear design for all-day wear

Target Audience: Music lovers and frequent travelers looking for a premium audio experience without distractions.

Use Cases:
1. Commuting on public transportation
2. Studying in a n